In [3]:
import pandas as pd
import numpy as np
from __future__ import division

In [4]:
attr_w = ['A', 'A', 'A', 'B', 'B','B', 'A']
attr_x = ['C', 'C', 'D', 'D', 'C', 'E', 'D']
attr_y = [.1, .3, .8, .9, .2, .6, .7]
attr_z = [.1, .9, .8, .2, .8, .9, .1]

df = pd.DataFrame({'W': attr_w, 'X': attr_x, 'Y': attr_y, 'Z': attr_z})
categ_df = df[['W', 'X']]

In [5]:
df.head()

,W,X,Y,Z
0,A,C,0.1,0.1
1,A,C,0.3,0.9
2,A,D,0.8,0.8
3,B,D,0.9,0.2
4,B,C,0.2,0.8


In [10]:
unique_categ_vals = []
for col in categ_df.columns:
    unique_categ_vals += categ_df[col].unique().tolist()
    
indexes = unique_categ_vals
columns = unique_categ_vals

In [7]:
categ_df['X'].unique().tolist()

['C', 'D', 'E']

In [9]:
unique_categ_vals

['A', 'B', 'C', 'D', 'E']

In [12]:
df_ = pd.DataFrame(index=indexes, columns=columns)

In [13]:
df_.head()

,A,B,C,D,E
A,NaN,NaN,NaN,NaN,NaN
B,NaN,NaN,NaN,NaN,NaN
C,NaN,NaN,NaN,NaN,NaN
D,NaN,NaN,NaN,NaN,NaN
E,NaN,NaN,NaN,NaN,NaN


In [33]:
df_ = pd.DataFrame(index=indexes, columns=columns)

def get_col_name(val):
    mapping = {'W': ['A', 'B'], 'X': ['C', 'D', 'E']}
    for k, v in mapping.iteritems():
        if val in mapping[k]:
            return(k)
        
# df.loc[index, 'COL_NAME'] = x
offset = 0
for index in indexes:
    offset_columns = columns[offset:]
    offset +=1
    for column in offset_columns:
        n = len(df[(df[get_col_name(index)] == index) & (df[get_col_name(column)] == column) ])
        df_.loc[index, column] = n
        
df_.fillna(0, inplace=True)

In [34]:
df_.head()

,A,B,C,D,E
A,4,0,2,2,0
B,0,3,1,1,1
C,0,0,3,0,0
D,0,0,0,3,0
E,0,0,0,0,1


4

In [37]:
df_distances = pd.DataFrame(index=indexes, columns=columns)

offset = 1
for index in indexes:
    offset_columns = columns[offset:]
    offset +=1
    for column in offset_columns:
        mxy = df_.loc[index, column]
        mx = df_.loc[index, index]
        my = df_.loc[column, column]
        dxy = mxy/(mx + my - mxy)
        df_distances.loc[index, column] = dxy

In [38]:
df_distances.head()

,A,B,C,D,E
A,NaN,0,0.4,0.4,0
B,NaN,NaN,0.2,0.2,0.333333
C,NaN,NaN,NaN,0,0
D,NaN,NaN,NaN,NaN,0
E,NaN,NaN,NaN,NaN,NaN


In [141]:
def within_group_variance(df, base_attr_col, numerical_cols):
    base_items = df[base_attr_col].unique().tolist()
    best_wgv = {'wgv': float('Inf')}
    for numeric_col in numeric_cols:
        ss = 0
        for base_item in base_items:
            sub_df = df[df[base_attr_col] == base_item]
            var = sub_df[numeric_col].var()
            var = 0 if np.isnan(var) else var
            ss += var

        if  ss < best_wgv['wgv']:
            best_wgv['wgv'] = ss
            best_wgv['attr_col'] = numeric_col
    return(best_wgv)
            
numeric_cols = ['Y', 'Z']
base_attr_col = 'X'
best = within_group_variance(df, base_attr_col, numeric_cols)


In [84]:
best_wgv

{'attr_col': 'Y', 'wgv': 0.020000000000000004}

In [91]:
df2 = df.groupby('X').agg({"Y": 'mean'}).reset_index().rename(columns={'Y': "numeric_X"})

In [92]:
df1 = df.copy()
df1.merge(df2, how='inner', on='X')

,W,X,Y,Z,numeric_X
0,A,C,0.1,0.1,0.2
1,A,C,0.3,0.9,0.2
2,B,C,0.2,0.8,0.2
3,A,D,0.8,0.8,0.8
4,B,D,0.9,0.2,0.8
5,A,D,0.7,0.1,0.8
6,B,E,0.6,0.9,0.6


In [71]:
class TMCMFeatureEng(object):
    def __init__(self, df, categ_cols, numeric_cols):
        self.df = df.copy()
        self.categ_cols = categ_cols
        self.numeric_cols = numeric_cols
        
    def _preprocessing(self):
        self.df[self.numeric_cols] = self.df[self.numeric_cols].apply(lambda x: 
                                                                      (x.astype(float) - min(x))/(max(x)-min(x)), 
                                                                      axis=0)
    
    def _create_matrix(self, indexes, columns):
        return(pd.DataFrame(index=indexes, columns=columns))
    
    def _get_col_name(self, val):
        for k, v in self.categ_mapping.iteritems():
            if val in self.categ_mapping[k]:
                return(k)         
    
    def set_categ(self):
        self.categ_vals = []
        self.categ_mapping = {}
        counter = 0
        for col in self.categ_cols:
            unique_vals = self.df[col].unique().tolist()
            self.categ_vals += unique_vals
            self.categ_mapping[col] = unique_vals
            if len(unique_vals) > counter:
                self.base_items = unique_vals
                self.base_col = col
            
    def fill_co_occurence_matrix(self):
        offset = 0
        for index in self.co_occurrence_matrix.index:
            offset_columns = self.co_occurrence_matrix.columns[offset:]
            offset +=1
            for column in offset_columns:
                i_col = self._get_col_name(index)
                c_col = self._get_col_name(column)
                n = len(self.df[ (self.df[i_col] == index) & (self.df[c_col] == column) ])
                self.co_occurrence_matrix.loc[index, column] = n
#         self.co_occurrence_matrix.fillna(0, inplace=True)
    
    def fill_distance_matrix(self):
        offset = 1
        for index in self.distance_matrix.index:
            offset_columns = self.distance_matrix.columns[offset:]
            offset +=1
            for column in offset_columns:
                mxy = self.co_occurrence_matrix.loc[index, column]
                mx = self.co_occurrence_matrix.loc[index, index]
                my = self.co_occurrence_matrix.loc[column, column]
                dxy = mxy/(mx + my - mxy)
                self.distance_matrix.loc[index, column] = dxy
#         self.distance_matrix.fillna(0, inplace=True)
    
    def _get_best_attr_within_group_variance(self):
        best_wgv = {'wgv': float('Inf')}
        for numeric_col in self.numeric_cols:
            ss = 0
            for base_item in self.base_items:
                sub_df = self.df[self.df[self.base_col] == base_item]
                var = sub_df[numeric_col].var()
                var = 0 if np.isnan(var) else var
                ss += var

            if  ss < best_wgv['wgv']:
                best_wgv['wgv'] = ss
                best_wgv['attr_col'] = numeric_col
                
        return(best_wgv)
    
    def calculate_value_for_categ(self):
        best_wgv = self._get_best_attr_within_group_variance()
        
        # CALCULATE FOR BASE COL
        df2 = self.df.groupby(self.base_col).agg({best_wgv['attr_col']: 'mean'}).reset_index()
        df2 = df2.rename(columns={best_wgv['attr_col']: "numeric_{}".format(self.base_col)})
#         self.teste = df2
        self.df = self.df.merge(df2, how='inner', on=self.base_col)
        
        #CALCULATE FOR REMAINING CATEG COLS
        for col, values in self.categ_mapping.iteritems():
            if col != self.base_col:
                self.df['numeric_{}'.format(col)] = np.nan
                for value in values: # unique values [A, B]
                    v = sum([self.distance_matrix.loc[value, item] * 
                             df2.loc[df2[self.base_col] == item, 'numeric_{}'.format(self.base_col)].iloc[0] 
                             for item in self.base_items])
                    self.df.loc[self.df[col] == value, 'numeric_{}'.format(col)] = v
    
    def transform(self, normalize=False):
        if normalize == True:
            self._preprocessing()
            
        self.set_categ()
        self.co_occurrence_matrix = self._create_matrix(self.categ_vals, self.categ_vals)
        self.distance_matrix = self._create_matrix(self.categ_vals, self.categ_vals)
        self.fill_co_occurence_matrix()
        self.fill_distance_matrix()
        self.calculate_value_for_categ()
        return(self.df)

In [69]:
obj = TMCMFeatureEng(df, ['W', 'X'], ['Y', 'Z'])
transformed_df = obj.transform()

In [70]:
transformed_df.head(10)

,W,X,Y,Z,numeric_X,numeric_W
0,A,C,0.1,0.1,0.2,0.4
1,A,C,0.3,0.9,0.2,0.4
2,B,C,0.2,0.8,0.2,0.4
3,A,D,0.8,0.8,0.8,0.4
4,B,D,0.9,0.2,0.8,0.4
5,A,D,0.7,0.1,0.8,0.4
6,B,E,0.6,0.9,0.6,0.4


In [28]:
sum([obj.distance_matrix.loc['B', item] * obj.teste.loc[obj.teste['X'] == item, 'numeric_X'].iloc[0] for item in obj.base_items])

0.4

In [51]:
obj.df.loc[obj.df['W'] == 'A', 'numeric_{}'.format(obj.base_col)]

0    0.4
1    0.4
3    0.4
5    0.4
Name: numeric_X, dtype: float64

In [37]:
obj.teste.loc[obj.teste['X'] == 'E', 'numeric_X'].iloc[0] 

0.6

In [38]:
(0.2*0.2) + (0.2*0.8) + (0.3333333*0.6)

0.39999998000000003